In [ ]:
import os
import json
import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras import models, layers, optimizers

DATASET_PATH = "Data/genres_original"
JSON_PATH = "data.json"
SAMPLE_RATE = 22050
DURATION = 30 # seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

In [ ]:
def prepare_dataset(dataset_path, json_path, n_mfcc=13, hop_length=512, n_fft=2048):
    data = {
        "mapping": [],
        "mfcc": [],
        "labels": []
    }
    num_samples_per_segment = int(SAMPLES_PER_TRACK / 10)
    print("Extracting features from dataset...")

    for i, (dirpath, _, filenames) in enumerate(os.walk(dataset_path)):
        if dirpath is not dataset_path:
            genre = os.path.basename(dirpath)
            data["mapping"].append(genre)
            print(f"\nProcessing: {genre}")

            for f in filenames:
                file_path = os.path.join(dirpath, f)
                try:
                    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
                except Exception as e:
                    print(f"Could not load {file_path}: {e}")
                    continue

                for s in range(10): # Process 10 segments per track
                    start_sample = num_samples_per_segment * s
                    end_sample = start_sample + num_samples_per_segment
                    mfcc = librosa.feature.mfcc(y=signal[start_sample:end_sample], sr=sr, n_fft=n_fft, n_mfcc=n_mfcc, hop_length=hop_length)
                    mfcc = mfcc.T

                    if len(mfcc) == 130:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i - 1)
                        print(".", end="")

    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)
    print("\n\nData extraction complete and saved to data.json.")

In [ ]:
# NOTE: Only run this cell once to create the data.json file.
prepare_dataset(DATASET_PATH, JSON_PATH)

In [ ]:
with open(JSON_PATH, "r") as fp:
    data = json.load(fp)

X = np.array(data["mfcc"])
y = np.array(data["labels"])
X = X[..., np.newaxis] # Add channel dimension for CNN

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def build_model(input_shape):
    model = models.Sequential(name="Music_Genre_Classifier")
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(10, activation='softmax'))
    return model

input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
model = build_model(input_shape)
optimizer = optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=50)

In [ ]:
model.save("music_genre_classifier.keras")
print("\nModel trained and saved as music_genre_classifier.keras")

In [ ]:
fig, axs = plt.subplots(2, figsize=(10, 10))
axs[0].plot(history.history['accuracy'], label='train accuracy')
axs[0].plot(history.history['val_accuracy'], label='test accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].legend(loc='lower right')
axs[0].set_title('Accuracy eval')
axs[1].plot(history.history['loss'], label='train error')
axs[1].plot(history.history['val_loss'], label='test error')
axs[1].set_ylabel('Error')
axs[1].set_xlabel('Epoch')
axs[1].legend(loc='upper right')
axs[1].set_title('Error eval')
plt.show()